In [22]:
from sys import stdout
 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
 
from scipy.signal import savgol_filter
 
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale 

from keras.models import Sequential
from keras.layers import Dense

from progressbar import ProgressBar

In [26]:

#import training datadet
#import internal data
inter = pd.ExcelFile('own LC data_v2.xlsx')
inter = pd.read_excel(inter,'Comb1')
#print(inter.info())
comb = inter.dropna(axis=0)
#print(comb.info())
comb = comb[comb["tR (min)"] != "?"]
#preprocess
for col in comb.columns[1:]:
    if (comb[col] == 0).all():
        comb = comb.drop([col], axis = 1)
comb = comb.reset_index(drop=True)
X_CAL = comb.drop(['Project','Column','Gnum','tR (min)'], axis=1)
print(X_CAL.info())
print(X_CAL.iloc[:,0:12])
y = {'tR (min)': [round(x,4) for x in comb['tR (min)']]}
y = pd.DataFrame(comb, columns = ['tR (min)'])
print(y)
print(np.unique(comb['Column']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1790 entries, 0 to 1789
Columns: 141 entries, H to fsulfone
dtypes: float64(100), int64(41)
memory usage: 1.9 MB
None
         H    S*     A     B  C (ph=2.8)  C (ph=7.0)  EB retention factor  \
0     1.00  0.02 -0.09  0.00        0.14        0.09                  7.9   
1     1.00  0.02 -0.09  0.00        0.14        0.09                  7.9   
2     1.00  0.02 -0.09  0.00        0.14        0.09                  7.9   
3     1.00  0.02 -0.09  0.00        0.14        0.09                  7.9   
4     1.00  0.02 -0.09  0.00        0.14        0.09                  7.9   
...    ...   ...   ...   ...         ...         ...                  ...   
1785  1.09  0.05  0.01 -0.04        0.03        0.01                  8.4   
1786  1.09  0.05  0.01 -0.04        0.03        0.01                  8.4   
1787  1.09  0.05  0.01 -0.04        0.03        0.01                  8.4   
1788  1.09  0.05  0.01 -0.04        0.03        0.01          

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def HPLC_COLUMN_TRANSFER_SEARCH(X_CAL, Y_CAL, ind_table, COL_A, COL_B):
    temp_A = ind_table[ind_table['Column'] == COL_A].index
    temp_B = ind_table[ind_table['Column'] == COL_B].index

    Gnum_A = ind_table.loc[temp_A, 'Gnum'].values
    Gnum_B = ind_table.loc[temp_B, 'Gnum'].values
    
    X_CAL_A = X_CAL[temp_A, :]
    Y_CAL_A = Y_CAL[temp_A, :]

    X_CAL_B = X_CAL[temp_B, :]
    Y_CAL_B = Y_CAL[temp_B, :]

    UNIQUE_LIST = np.unique(np.concatenate((Gnum_A, Gnum_B)))
    

    X_SEL_A = np.empty((0, X_CAL.shape[1]))
    Y_SEL_A = np.empty((0, Y_CAL.shape[1]))
    X_SEL_B = np.empty((0, X_CAL.shape[1]))
    Y_SEL_B = np.empty((0, Y_CAL.shape[1]))
    Gnum = np.empty((0,), dtype=str)
    
    for unique_item in UNIQUE_LIST:

        ind_A = np.where(Gnum_A == unique_item)[0]
        ind_B = np.where(Gnum_B == unique_item)[0]

        if ind_A.size == 0 or ind_B.size == 0:
            continue
        elif ind_A.size == ind_B.size:
            
            for k in range(len(ind_A)):
                # column 11 is the pH
                print('haha')
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Gnum = np.append(Gnum, unique_item)
        elif ind_A.size < ind_B.size:
            for k in range(len(ind_A)):
                temp = X_CAL_A[ind_A[k], 11] - X_CAL_B[ind_B, 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, X_CAL_A[ind_A[k], :]])
                    Y_SEL_A = np.vstack([Y_SEL_A, Y_CAL_A[ind_A[k], :]])
                    X_SEL_B = np.vstack([X_SEL_B, np.mean(X_CAL_B[ind_B[ind], :], axis=0)])
                    Y_SEL_B = np.vstack([Y_SEL_B, np.mean(Y_CAL_B[ind_B[ind], :], axis=0)])
                    Gnum = np.append(Gnum, unique_item)
        elif ind_A.size > ind_B.size:
            
            for k in range(len(ind_B)):
                temp = X_CAL_A[ind_A, 11] - X_CAL_B[ind_B[k], 11]

                ind = np.where(temp == 0)
                ind = np.asarray(ind).astype('int16')

                if ind.size != 0:
                    X_SEL_A = np.vstack([X_SEL_A, np.mean(X_CAL_A[ind_A[ind], :], axis=0)])
                    Y_SEL_A = np.vstack([Y_SEL_A, np.mean(Y_CAL_A[ind_A[ind], :], axis=0)])
                    X_SEL_B = np.vstack([X_SEL_B, X_CAL_B[ind_B[k], :]])
                    Y_SEL_B = np.vstack([Y_SEL_B, Y_CAL_B[ind_B[k], :]])
                    Gnum = np.append(Gnum, unique_item)
    
    RESULTS = {
        'X_SEL_A': X_SEL_A,
        'X_SEL_B': X_SEL_B,
        'Y_SEL_A': Y_SEL_A,
        'Y_SEL_B': Y_SEL_B,
        'Gnum': Gnum
    }

    if len(Y_SEL_A) == len(Y_SEL_B):
        plt.plot(Y_SEL_A, Y_SEL_B, 'ro')
        plt.xlabel(f'measured Rt on {COL_A} (min)')
        plt.ylabel(f'measured Rt on {COL_B} (min)')
        plt.show()
    
    return RESULTS

In [25]:
ind_table=pd.DataFrame(comb, columns = ['Project','Column','Gnum'])
X_CAL = comb.drop(['Project','Column','Gnum','tR (min)'], axis=1)
Y_CAL = np.asarray(y).astype('float32')
X_CAL = np.asarray(X_CAL).astype('float32')
RESULTS=HPLC_COLUMN_TRANSFER_SEARCH(X_CAL, Y_CAL, ind_table, 'ACE AQ','BEH C18 UPLC')
print(RESULTS['X_SEL_A'].shape)
print(RESULTS['X_SEL_B'].shape)
print(RESULTS['Gnum'].shape)

['G-044378.1-1' 'G-056787.1-1' 'G-057410' 'G-057521.1-1' 'G-057525.1-1'
 'G-060097.1-1' 'G-060098.1-1' 'G00050939' 'G00505032' 'G00587678'
 'G00587744' 'G00590286' 'G01524148' 'G01524149' 'G01577625' 'G01577626'
 'G02298192' 'G02599853' 'G02629352' 'G02771827' 'G02771828' 'G02771829'
 'G02773237' 'G02776497' 'G02792134' 'G02854057' 'G02859359' 'G02859750'
 'G02941983' 'G02947024' 'G02950274' 'G02950579' 'G02950580' 'G02950581'
 'G02950582' 'G02950585' 'G02950862' 'G02950863' 'G02950865' 'G02950868'
 'G02950873' 'G02967907' 'G03044276' 'G03047472' 'G03050248' 'G03050251'
 'G03071777' 'G03076166' 'G03085879']
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
haha
(82, 141)
(87, 141)
(82,)
